In [7]:
import cv2
import numpy as np
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
from mediapipe.framework.formats import landmark_pb2

In [268]:
model_path = "C:\\Users\\kyles\\Desktop\\CS5100\\Final_Project\\pose_landmarker_full.task"
video_source = "C:\\Users\\kyles\\Desktop\\CS5100\\Final_Project\\Project_Data\\PEN26.mp4"
min_pose_detection_confidence = 0.5
min_pose_presence_confidence = 0.5
min_tracking_confidence = 0.5
num_poses = 2
last_timestamp_ms = 0

In [269]:
def draw_landmarks_on_image(rgb_image, detection_result):
    pose_landmarks_list = detection_result.pose_landmarks
    annotated_image = np.copy(rgb_image)

    # Loop through the detected poses to visualize.
    for idx in range(len(pose_landmarks_list)):
        pose_landmarks = pose_landmarks_list[idx]

        pose_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
        pose_landmarks_proto.landmark.extend([
            landmark_pb2.NormalizedLandmark(
                x=landmark.x,
                y=landmark.y,
                z=landmark.z) for landmark in pose_landmarks
        ])
        mp.solutions.drawing_utils.draw_landmarks(
            annotated_image,
            pose_landmarks_proto,
            mp.solutions.pose.POSE_CONNECTIONS,
            mp.solutions.drawing_styles.get_default_pose_landmarks_style())
    return annotated_image

In [270]:
base_options = python.BaseOptions(model_asset_path=model_path)
options = vision.PoseLandmarkerOptions(base_options=base_options,
                                      running_mode=vision.RunningMode.VIDEO,
                                      num_poses=num_poses,
                                      min_pose_detection_confidence=min_pose_detection_confidence,
                                      min_pose_presence_confidence=min_pose_presence_confidence,
                                      min_tracking_confidence=min_tracking_confidence,
                                      output_segmentation_masks=False)

In [ ]:
'''

1) Resize and convert frame from RGB to BGR
2) Detect poses
    - z represents distance -- we want closer (penalty taker) pose
    - append relevant data to lists for feature extraction
3) Display image with pose draw onto it

'''

In [271]:
pose_data = []
mp_data = []
pose_draw_data = []
time_stamps = []


with vision.PoseLandmarker.create_from_options(options) as landmarker:
    # Use OpenCV’s VideoCapture to start capturing from the webcam.
    cap = cv2.VideoCapture(video_source)
    # Create a loop to read the latest frame from the camera using VideoCapture#read()
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            
            break
            
            
        image = cv2.resize(image, None, fx=.5, fy=.5)
        image = image[100:500, 250:500]
        # Convert the frame received from OpenCV to a MediaPipe’s Image object.
        mp_image = mp.Image(
            image_format=mp.ImageFormat.SRGB,
            data=cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        timestamp_ms = int(cv2.getTickCount() / cv2.getTickFrequency() * 1000)
        result = landmarker.detect_for_video(mp_image, timestamp_ms)
        pose_data.append(result.pose_world_landmarks)
        annotated_image = draw_landmarks_on_image(mp_image.numpy_view(), result)
        
        if result.pose_world_landmarks:
            if result.pose_world_landmarks[0][0].z > -.3 or result.pose_world_landmarks[0][1].z > -.3:
                # get frame_ms -- append to list
                frame_ms = cap.get(cv2.CAP_PROP_POS_MSEC)
                time_stamps.append(frame_ms)
                # draw frames
                pose_draw_data.append(result)
                mp_data.append(mp_image)
                
            
        
        cv2.imshow("MediaPipe Pose Landmark", cv2.cvtColor(annotated_image, cv2.COLOR_BGR2RGB))

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()

In [279]:
lower_half_data = []
print(type(pose_draw_data[0]))
pose_draw_data[0].pose_landmarks[0][23::]

for i in range(len(pose_draw_data)):
    for each in pose_draw_data[i].pose_landmarks:
        lower_half_data.append(each[23::])

len(lower_half_data)


<class 'mediapipe.tasks.python.vision.pose_landmarker.PoseLandmarkerResult'>


82

In [285]:
num = 0

for i in range(len(pose_draw_data)):
    num += len(pose_draw_data[i].pose_landmarks)
    print(num)
    
num

1
2
3
4
5
7
9
11
13
15
17
19
21
23
25
27
29
31
33
35
36
38
40
42
44
46
48
49
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
80
82


82

In [283]:
for i in range(len(pose_draw_data)):

    test_output = draw_landmarks_on_image(mp_data[i].numpy_view(), pose_draw_data[i])
    while True:
        cv2.imshow('result', test_output)
        if cv2.waitKey(1) & 0xFF == ord('q'):
                break
cv2.destroyAllWindows()


In [201]:
pose_draw_data[0].pose_landmarks[1]

IndexError: list index out of range